In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from yellowbrick.target import FeatureCorrelation
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PowerTransformer
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where
import warnings
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns',None)
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
df=pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')
df.head()

In [ ]:
df.shape

In [ ]:
#checking the datatypes of the columns
df.info()

In [ ]:
df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d')   #changing the datatype of the Date column from object to DateTime

In [ ]:
#checking percentage of null values in each column
(df.isnull().sum()/len(df))*100

In [ ]:
#dropping the rows of the column having less than 5% null values
df=df.dropna(subset=['MinTemp','MaxTemp','Rainfall','WindDir3pm','WindSpeed3pm','Humidity9am','Humidity3pm','Temp9am','Temp3pm','RainToday','RainTomorrow'],axis=0)

In [ ]:
df.isnull().sum()

In [ ]:
df.skew()

In [ ]:
# replacing the null values of Pressure9am,Pressure3pm, Cloud9am, Cloud3pm with mean as the skewness is very less
for i in ['Pressure9am','Pressure3pm', 'Cloud9am', 'Cloud3pm']:
    df[i].fillna(value=df[i].mean(),inplace=True)

In [ ]:
# replacing the null values of Evaporation,Sunshine,WindGustSpeed,WindSpeed9am with median as they have high skewness
for i in ['Evaporation','Sunshine','WindGustSpeed','WindSpeed9am']:
    df[i].fillna(value=df[i].median(),inplace=True)

In [ ]:
# replacing the null values of WindGustDir, WindDir9am with the mode as it is a categorical data
for i in ['WindGustDir', 'WindDir9am']:
    df[i]=df[i].fillna(df[i].mode()[0])

In [ ]:
#the data is now free from null values
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
plt.figure(figsize=(18,15))
sns.heatmap(df.corr(),annot=True)
plt.show()

In [ ]:
#Count plot for the distribution of the target variable

In [ ]:
sns.countplot(df['RainTomorrow'])
plt.show()

In [ ]:
#the target variable is highly imbalanced as there are hue number of No as compared to Yes

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(121)
sns.countplot(df['RainToday'])
plt.subplot(122)
sns.countplot(df['RainTomorrow'])
plt.show()

In [ ]:
#There seems to be high correlation between the rain today and the rain tomorrow columns as the count of rainfall and
#no rainfall is nearly same

In [ ]:
plt.figure(figsize=(18,5))
plt.subplot(121)
sns.countplot(df['WindDir9am'],hue=df['RainTomorrow'])
plt.subplot(122)
sns.countplot(df['WindDir3pm'],hue=df['RainTomorrow'])
plt.show()

In [ ]:
#In the morning if the wind direction is North , we have chances of rainfall and no rainfall, as the ,maximum count of rainfall
#and no rainfall is when the wind is blowing in the north direction, which means windDir9am is not a good predictor

#In the evening the highest chance of rainfall is when the wind is blowing south, and the highest chance of no rainfall is
#when the wind is blowing in the west direction, so WindDir3pm is a good predictor

In [ ]:
plt.figure(figsize=(16,6))
sns.countplot(df['RainTomorrow'],hue=df['WindGustDir'])
plt.show()

In [ ]:
#With respect to the WindGustDir column the maximum chances of rainfall is when the wind is blowing in West Direction
#and the maximum chances of no rainfall is also when the wind is blowing in west direction so we cannot classify if it will 
#rain or not using the WindGustDir .

In [ ]:
plt.figure(figsize=(18,5))
plt.subplot(121)
sns.barplot(df['RainTomorrow'],df['Cloud3pm'])
plt.subplot(122)
sns.barplot(df['RainTomorrow'],df['Cloud9am'])
plt.show()

In [ ]:
#If clouds are more then there are high chances of rain

In [ ]:
plt.figure(figsize=(18,5))
plt.subplot(121)
sns.barplot(df['RainTomorrow'],df['Pressure3pm'])
plt.subplot(122)
sns.barplot(df['RainTomorrow'],df['Pressure9am'])
plt.show()

In [ ]:
#there seems to be no correlation of the pressure on the rain but this might be due to presence of outliers

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(df['Date'],df['Humidity3pm'],label='Humidity3pm',linewidth=1)
sns.lineplot(df['Date'],df['Humidity9am'],label='Humidity9am',linewidth=1)
plt.ylabel('Humidity')
plt.show()

In [ ]:
#The humidity in 2008 is high as compared to 2017
#The humidity at 3pm is always high as compared to humidity at 9am
#The humidity is high in the mid of the year

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(df['Date'],df['WindSpeed3pm'],label='WindSpeed3pm',linewidth=1)
sns.lineplot(df['Date'],df['WindSpeed9am'],label='WindSpeed9am',linewidth=1)
plt.ylabel('WindSpeed')
plt.show()

In [ ]:
#The windspeed has been constant from 2008-2017
#The wind at 3pm is always high as compared to speed at 9am

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(df['Date'],df['Pressure3pm'],label='Pressure3pm',linewidth=2)
sns.lineplot(df['Date'],df['Pressure9am'],label='Pressure9am',linewidth=2)
plt.ylabel('Pressure')
plt.show()

In [ ]:
#The pressure is almost same during the entire time of the day

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(df['Date'],df['Cloud3pm'],label='Cloud3pm',linewidth=1)
sns.lineplot(df['Date'],df['Cloud9am'],label='Cloud9am',linewidth=1)
plt.ylabel('Clouds')
plt.show()

In [ ]:
#The clouds all almost same during the entire time of the day

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(df['Date'],df['Temp3pm'],label='Temp3pm',linewidth=1)
sns.lineplot(df['Date'],df['Temp9am'],label='Temp9am',linewidth=1)
plt.ylabel('Temperature')
plt.show()

In [ ]:
#the temperature is low at 9pm and the temperature is high at 3pm
#the temperature is lowest in the mid of the year

In [ ]:
#label encoding all the categorical data
for i in df.loc[:,df.dtypes=='object'].columns:
    le=LabelEncoder()
    df[i]=le.fit_transform(df[i])

# Modelling data with different algorithms to find the best model

In [ ]:
X=df.drop(['RainTomorrow','Date'],axis=1)
y=df['RainTomorrow']

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.25,random_state=2)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',RandomForestClassifier(n_estimators=100,random_state=4)),
))
pipe.fit(xtrain,ytrain)
rftest_score=pipe.score(xtest,ytest)
rftrain_score=pipe.score(xtrain,ytrain)
print('Test score',rftest_score)
print('Train score',rftrain_score)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('gbc',GradientBoostingClassifier(n_estimators=100)),
))
pipe.fit(xtrain,ytrain)
gbtest_score=pipe.score(xtest,ytest)
gbtrain_score=pipe.score(xtrain,ytrain)
print('Test score',gbtest_score)
print('Train score',gbtrain_score)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('lr',LogisticRegression()),
))
pipe.fit(xtrain,ytrain)
lrtest_score=pipe.score(xtest,ytest)
lrtrain_score=pipe.score(xtrain,ytrain)
print('Test score',lrtest_score)
print('Train score',lrtrain_score)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('dt',DecisionTreeClassifier(max_depth=8)),
))
pipe.fit(xtrain,ytrain)
dttest_score=pipe.score(xtest,ytest)
dttrain_score=pipe.score(xtrain,ytrain)
print('Test score',dttest_score)
print('Train score',dttrain_score)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('ada',AdaBoostClassifier(n_estimators=10)),
))
pipe.fit(xtrain,ytrain)
adgbtest_score=pipe.score(xtest,ytest)
adgbtrain_score=pipe.score(xtrain,ytrain)
print('Test score',adgbtest_score)
print('Train score',adgbtrain_score)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('xg',XGBClassifier()),
))
pipe.fit(xtrain,ytrain)
xgtest_score=pipe.score(xtest,ytest)
xgtrain_score=pipe.score(xtrain,ytrain)
print('Test score',xgtest_score)
print('Train score',xgtrain_score)

In [ ]:
l1=['DecisionTree','RandomForest','GradientBoost','AdaBoost','XGBoost','LogisticRegression']
l2=[dttest_score,rftest_score,gbtest_score,adgbtest_score,xgtest_score,lrtest_score]
l3=[dttrain_score,rftrain_score,gbtrain_score,adgbtrain_score,xgtrain_score,lrtrain_score]

In [ ]:
score_df=pd.DataFrame({'Models':l1,'Test Score':l2,'Train Score':l3})

In [ ]:
score_df

In [ ]:
# RandomForest performs the best here

### Applying cross validation to find the best model

In [ ]:
score_grad=cross_val_score(X=X,y=y,cv=10,scoring='accuracy',n_jobs=-1,estimator=GradientBoostingClassifier())

In [ ]:
score_grad

In [ ]:
score_grad.mean()

In [ ]:
score_rf=cross_val_score(X=X,y=y,cv=10,scoring='accuracy',n_jobs=-1,estimator=RandomForestClassifier())

In [ ]:
score_rf

In [ ]:
score_rf.mean()

In [ ]:
score_xg=cross_val_score(X=X,y=y,cv=10,scoring='accuracy',n_jobs=-1,estimator=XGBClassifier())

In [ ]:
score_xg

In [ ]:
score_xg.mean()

In [ ]:
# From the above scores we get to know that random forest performs the best

In [ ]:
# Performing grid search to find the best hyperparameters

In [ ]:
params=[{'n_estimators':[10,20,50,10,100,500],'max_depth':[3,5,7,9],'criterion':['entropy','gini']}]
grid=GridSearchCV(estimator=RandomForestClassifier(),cv=10,scoring='accuracy',param_grid=params)

In [ ]:
grid.fit(xtrain,ytrain)

In [ ]:
grid.best_estimator_

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',RandomForestClassifier(n_estimators=500,random_state=9)),
))
pipe.fit(xtrain,ytrain)
rf_score=pipe.score(xtest,ytest)
rf_score

In [ ]:
pipe.score(xtrain,ytrain)

In [ ]:
#The model has an accuracy of 86.2% but it is an overfit model as the training score is 1

# Removing the outliers in the data

In [ ]:
df=pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')
df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d')   #changing the datatype of the Date column from object to DateTime
df=df.dropna(subset=['MinTemp','MaxTemp','Rainfall','WindDir3pm','WindSpeed3pm','Humidity9am','Humidity3pm','Temp9am','Temp3pm','RainToday','RainTomorrow'],axis=0)

In [ ]:
# replacing the null values of Pressure9am,Pressure3pm, Cloud9am, Cloud3pm with mean as the skewness is very less
for i in ['Pressure9am','Pressure3pm', 'Cloud9am', 'Cloud3pm']:
    df[i].fillna(value=df[i].mean(),inplace=True)

In [ ]:
# replacing the null values of Evaporation,Sunshine,WindGustSpeed,WindSpeed9am with median as they have high skewness
for i in ['Evaporation','Sunshine','WindGustSpeed','WindSpeed9am']:
    df[i].fillna(value=df[i].median(),inplace=True)

In [ ]:
# replacing the null values of WindGustDir, WindDir9am with the mode as it is a categorical data
for i in ['WindGustDir', 'WindDir9am']:
    df[i]=df[i].fillna(df[i].mode()[0])

In [ ]:
df.shape

In [ ]:
num_col=df.loc[:,df.dtypes!='object'].drop('Date',axis=1).columns

In [ ]:
num_col

In [ ]:
for i in num_col:
    q1=df[i].quantile(.25)
    q3=df[i].quantile(.75)
    iqr=q3-q1
    ll=q1-1.5*iqr
    ul=q3+1.5*iqr
    a=df[(df[i]<ll) | (df[i]>ul)].index
    df.drop(a,axis=0,inplace=True)
    
#removing outliers with iqr method

In [ ]:
df.shape

In [ ]:
for i in df.loc[:,df.dtypes=='object'].columns:
    le=LabelEncoder()
    df[i]=le.fit_transform(df[i])

In [ ]:
X1=df.drop(['RainTomorrow','Date'],axis=1)
y1=df['RainTomorrow']

In [ ]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split(X1,y1,test_size=0.25,random_state=5)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('lr',LogisticRegression()),
))
pipe.fit(xtrain1,ytrain1)
lrtest_score=pipe.score(xtest1,ytest1)
lrtrain_score=pipe.score(xtrain1,ytrain1)
print('Test score',lrtest_score)
print('Train score',lrtrain_score)
#the model has an accuracy of 88.24% performs better after removing outliers

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('dt',DecisionTreeClassifier()),
))
pipe.fit(xtrain1,ytrain1)
dttest_score=pipe.score(xtest1,ytest1)
dttrain_score=pipe.score(xtrain1,ytrain1)
print('Test score',dttest_score)
print('Train score',dttrain_score)

#the model has an accuracy of 82.97% performs better after removing outliers

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',RandomForestClassifier()),
))
pipe.fit(xtrain1,ytrain1)
rftest_score=pipe.score(xtest1,ytest1)
rftrain_score=pipe.score(xtrain1,ytrain1)
print('Test score',rftest_score)
print('Train score',rftrain_score)

#the model is giving an accuracy of 89.05% but it is an overfit model as the training score is very high

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',AdaBoostClassifier()),
))
pipe.fit(xtrain1,ytrain1)
adgbtest_score=pipe.score(xtest1,ytest1)
adgbtrain_score=pipe.score(xtrain1,ytrain1)
print('Test score',adgbtest_score)
print('Train score',adgbtrain_score)

#the model performs better than RandomForestClassifier and has an accuracy of 88.8%

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',GradientBoostingClassifier()),
))
pipe.fit(xtrain1,ytrain1)
gbtest_score=pipe.score(xtest1,ytest1)
gbtrain_score=pipe.score(xtrain1,ytrain1)
print('Test score',gbtest_score)
print('Train score',gbtrain_score)

#the model performs better than both RandomForestClassifier and adaboosting and has an accuracy of 89.13%

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',XGBClassifier()),
))
pipe.fit(xtrain1,ytrain1)
xgbtest_score=pipe.score(xtest1,ytest1)
xgbtrain_score=pipe.score(xtrain1,ytrain1)
print('Test score',xgbtest_score)
print('Train score',xgbtrain_score)

#the model performs similar to gradientBoostingClassifier and better than all other models and has an accuracy of 89.15%
#but it is overfitted

In [ ]:
l1=['DecisionTree','RandomForest','GradientBoost','AdaBoost','XGBoost','LogisticRegression']
l2=[dttest_score,rftest_score,gbtest_score,adgbtest_score,xgtest_score,lrtest_score]
l3=[dttrain_score,rftrain_score,gbtrain_score,adgbtrain_score,xgtrain_score,lrtrain_score]

In [ ]:
newscore_df=pd.DataFrame({'Models':l1,'Test Score':l2,'Train Score':l3})

In [ ]:
newscore_df

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(newscore_df['Models'],newscore_df['Test Score'])
plt.show()

In [ ]:
#the models improved after removing outliers
#the gradiantboosting algorith gave the best results

# Feature selection

In [ ]:
feature_names = list(X1.columns)

In [ ]:
#Feature Correlation
visualizer = FeatureCorrelation(labels = feature_names)
visualizer.fit(X1, y1)
visualizer.poof()
plt.show()

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
sfs=SequentialFeatureSelector(estimator=XGBClassifier(),k_features=10,scoring='accuracy',forward=True)

In [ ]:
sfs.fit(xtrain1,ytrain1)

In [ ]:
sfs.k_feature_names_

In [ ]:
X2=df[['Location',
 'MaxTemp',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindDir3pm',
 'Humidity3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp3pm']]
y2=df['RainTomorrow']

In [ ]:
xtrain2,xtest2,ytrain2,ytest2=train_test_split(X2,y2,test_size=0.25,random_state=8)

In [ ]:
pipe=Pipeline((
('sc',StandardScaler()),
('pt',PowerTransformer()),
('rf',XGBClassifier()),
))
pipe.fit(xtrain2,ytrain2)
selective_testscore=pipe.score(xtest2,ytest2)
selective_trainscore=pipe.score(xtrain2,ytrain2)
print('Test Score',selective_testscore)
print('Training Score',selective_trainscore)


In [ ]:
#even on selecting top 10 features the model is not performing better
#The feature selection did not improve the model accuracy much

In [ ]:
# The best accuracy of 89.13% was given after removing the outliers and using the GradientBoost
# algorithm